# Scikit Learn - Resample
- - -

## Resample

- - -

Algunas técnicas y su idea:

- Upsampling

    Emplea un proceso de duplicar aleatoriamente las observaciones de la clase minoritaria para reforzarla.

- Downsampling

    Esta metodo implica eliminar observaciones al azar de la clase mayoritaria para evitar que su presencia domine el algoritmo de aprendizaje.

- - -

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
from IPython.display import set_matplotlib_formats
set_matplotlib_formats("retina")
import warnings
warnings.filterwarnings(action="ignore")

In [ ]:
from sklearn.utils import resample

- - -

In [ ]:
#Creamos datos sintéticos para trabajar con ellos
X_0 = pd.DataFrame(np.random.randint(0,60,size=(100, 2)), columns=['A','B'])
X_1 = pd.DataFrame(np.random.randint(40,100,size=(1000, 2)), columns=['A','B'])
X = X_0.append(X_1).reset_index(drop=True)

y_0 = pd.DataFrame(np.random.randint(0,1,size=(100, 1)), columns=['target'])
y_1 = pd.DataFrame(np.random.randint(1,2,size=(1000, 1)), columns=['target'])
y = y_0.append(y_1).reset_index(drop=True)
dataset = pd.concat([X,y],axis=1)
print(dataset.head())
print(dataset.tail())

In [ ]:
dataset["target"].value_counts()

In [ ]:
#Vemos que el dataset está muy desbalanceado
sns.countplot(x=dataset["target"]);

In [ ]:
sns.scatterplot(x=dataset["A"], y = dataset['B'], hue = dataset['target'])

#### Upsample Code Example
- - -

In [ ]:
clase_mayoritaria = dataset[dataset["target"] == 1]
clase_minoritaria = dataset[dataset["target"] == 0]

In [ ]:
#PRIMERA IDEA: básica
#Hacemos que la clase minoritaria tenga 1000 instancias, pero serán una copia de
#instancias que ya están en el dataset.
clase_minoritaria_upsampled = resample(clase_minoritaria, n_samples=1000, random_state=1)
print(clase_minoritaria_upsampled)

In [ ]:
upsampled_dataset = pd.concat([clase_mayoritaria, clase_minoritaria_upsampled])    
upsampled_dataset["target"].value_counts()                                   

In [ ]:
#A pesar de que hay más datos, si los printamos vemos que están unos encima de otros.
sns.scatterplot(x=upsampled_dataset["A"], y = upsampled_dataset['B'], hue = upsampled_dataset['target'])

#### Downsample Code Example

- - -

In [ ]:
clase_mayoritaria = dataset[dataset["target"] == 1]
clase_minoritaria = dataset[dataset["target"] == 0]

In [ ]:
#PRIMERA IDEA: básica
#Eliminamos muestras aleatorias de la clase mayoritaria para quedarnos con solamente 100 muestras.
clase_mayoritaria_downsampled = resample(clase_mayoritaria, n_samples=100, random_state=1)

In [ ]:
downsampled_dataset = pd.concat([clase_minoritaria, clase_mayoritaria_downsampled])  
downsampled_dataset["target"].value_counts() 
print(downsampled_dataset)                                    

In [ ]:
#Ahora vemos que hay muchas menos muestras
sns.scatterplot(x=downsampled_dataset["A"], y = downsampled_dataset['B'], hue = downsampled_dataset['target'])

- - -
https://scikit-learn.org/stable/modules/generated/sklearn.utils.resample.html
- - -

### Pero... ¿Podemos hacer algo más sofisticado? --> Data augmentation

- - -
- SMOTE (Synthetic Minority Over-sampling Technique)

    Es una técnica que consiste en sintetizar elementos para la clase minoritaria basados en los que ya existen.  
    Funciona seleccionando aleatoriamente un punto de la clase minoritaria y calculando los k vecinos más cercanos para este punto.  
    Los puntos sintéticos se agregan entre el punto elegido y sus vecinos.

In [ ]:
from imblearn.over_sampling import SMOTE
from collections import Counter

smote = SMOTE()
X = dataset.drop("target",1)
y = dataset["target"]
X_res, y_res = smote.fit_resample(X, y)
print(X_res)
print(Counter(y_res))

In [ ]:
#Analicemos cómo han quedado las muestras ahora.
#Comparad con el plot de upsample.
sns.scatterplot(x=X_res["A"], y = X_res['B'], hue = y_res)

- ADASYN

    Hace lo que SMOTE pero después de crear las nuevas muestras, agrega pequeños valores aleatorios para no estar correlacionadas linealmente, en otras palabras agrega dispersión.

In [ ]:
from imblearn.over_sampling import ADASYN
from collections import Counter

adasyn = ADASYN()
X_res2, y_res2 = adasyn.fit_resample(X, y)
print(X_res2)
print(Counter(y_res2))

In [ ]:
sns.scatterplot(x=X_res2["A"], y = X_res2['B'], hue = y_res2)

---
https://imbalanced-learn.org

- - -

## Class Weight

De forma predeterminada, el valor de class_weight = None, es decir, a ambas clases se les ha asignado el mismo peso.
También podemos darle como valor 'balanced' o podemos pasar un diccionario que contiene pesos manuales para ambas clases.

Cuando class_weights = 'balanced', el modelo asigna automáticamente los pesos de clase inversamente proporcionales a sus respectivas frecuencias.

### Dataset Pima Indians Diabetes
- - -

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

- - -

In [ ]:
diabetes = pd.read_csv("https://raw.githubusercontent.com/4data-lab/datasets/master/diabetes.csv", header=None)

In [ ]:
diabetes.columns = ["num_preg", "glucose_conc", "diastolic_bp", "thickness", "insulin", "bmi", "diab_pred", "age", "diabetes"]

In [ ]:
diabetes.head()

In [ ]:
sns.countplot(x=diabetes["diabetes"])

In [ ]:
X = diabetes.drop(["diabetes"], axis=1)
y = diabetes["diabetes"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.30, random_state=2)

#### Validación Score

La F1-score tiene en cuenta por igual la precisión y el recall:
 F1 = (2 * precision * recall) / (precision + recall)

In [ ]:
LR = LogisticRegression()
print("F1 - Datos de validation")
cross_val_score(LR, X_train, y_train, cv=5, scoring="f1").mean().round(2)

- - -

In [ ]:
#Añadimos el argumento class_weight
LR = LogisticRegression(class_weight="balanced")
print("F1 - Datos de validation")
cross_val_score(LR, X_train, y_train, cv=5, scoring="f1").mean().round(2)

- - -

Calcular el Class Weight:

In [ ]:
diabetes["diabetes"].value_counts()

In [ ]:
diabetes.shape

In [ ]:
número_de_muestras = diabetes.shape[0]

In [ ]:
número_de_muestras_clase_mayoritaria = diabetes["diabetes"].value_counts()[0]
número_de_muestras_clase_minoritaria = diabetes["diabetes"].value_counts()[1]
número_de_clases = diabetes["diabetes"].nunique()

In [ ]:
W0 = (número_de_muestras / (número_de_clases * número_de_muestras_clase_mayoritaria)).round(3)
W1 = (número_de_muestras / (número_de_clases * número_de_muestras_clase_minoritaria)).round(3)

In [ ]:
print("W0:", W0)
print("W1:", W1)

In [ ]:
LR = LogisticRegression(class_weight={0: 0.768, 1: 1.433})
print("F1 - Datos de validation")
cross_val_score(LR, X_train, y_train, cv=5, scoring="f1").mean().round(2)

#### Test Score

In [ ]:
LR.fit(X_train, y_train)
y_test_pred = LR.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
matriz_de_confusión = confusion_matrix(y_test, y_test_pred)
print(matriz_de_confusión)

In [ ]:
VN = matriz_de_confusión[0][0]
FP = matriz_de_confusión[0][1]
FN = matriz_de_confusión[1][0]
VP = matriz_de_confusión[1][1]

In [ ]:
print ("Verdaderos Negativos \t", VN)
print ("Falsos Positivos \t", FP)
print ("Falsos Negativos \t", FN)
print ("Verdaderos Positivos \t", VP)

#### ACCURACY (aka Exactitud):
Porcentaje de predicciónes correctas

In [ ]:
print(((VP+VN)/(VP+VN+FP+FN)).round(2))

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_test_pred).round(2)

- - -
| *Matriz Confusión*        | Predicción Negativa    | Predicción Positiva     |
| -------------             | :-------------:          | :-------------:           |
| **Observación Negativa** | Verdaderos Negativo (VN)  | Falsos Positivo (FP)      |
| **Observación Positiva** | Falsos Negativo (FN)      | Verdaderos Positivos (VP) |

#### ESPECIFICIDAD (aka Specificity):
Porcentaje de casos de observaciones negativas detectadas

In [ ]:
print((VN/(VN+FP)).round(2)) 

#### SENSIBILIDAD (aka Recall):  
Porcentaje de casos de observaciones positivas detectadas

In [ ]:
print((VP/(VP+FN)).round(2))

In [ ]:
from sklearn.metrics import recall_score
recall_score(y_test, y_test_pred).round(2)

- - -

#### PRECISIÓN (aka Precision): 
Porcentaje de predicciones de verdaderos positivos correctos

In [ ]:
print((VP/(VP+FP)).round(2))

In [ ]:
from sklearn.metrics import precision_score
precision_score(y_test, y_test_pred).round(2)

#### F1 (aka balanced F-score):  

Esta es otra métrica muy empleada porque nos resume la precisión y sensibilidad en una sola métrica por ello es de gran utilidad cuando la distribución de las clases está desbalanceada.

A diferencia de la exactitud, que se ve muy afectada por una gran cantidad de Verdaderos Negativos que en la mayoría de los casos no son de nuestro mayor interes, los Falsos Negativos y Falsos Positivos usualmente tienen impacto en nuestra solución. 

In [ ]:
Recall = (VP/(VP+FN)).round(2)

In [ ]:
Precision = (VP/(VP+FP)).round(2)

In [ ]:
print((2 * (Recall * Precision) / (Recall + Precision)).round(2))

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test, y_test_pred).round(2)

- - -

### Reporte de Clasificación

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test, y_test_pred))

- - -

    Support es el número de registros de cada clase en los datos del reporte de clasificación

In [ ]:
y_test.value_counts()

- - -